# Day10_2: A/B 테스트 실전 프로젝트 (A/B Test Practical Project) - 정답

## 📚 학습 목표

**Part 1: 프로젝트 기획 및 데이터 준비**
1. A/B 테스트 가설 설정 및 실험 설계하기
2. 표본 크기 및 실험 기간 계산하기
3. 다중 변수 A/B 테스트 데이터 탐색하기
4. 세그먼트별 데이터 전처리하기
5. 기초 통계량 및 그룹 간 비교하기

**Part 2: 분석 및 의사결정**
1. 통계적 검정 파이프라인 구축하기
2. Plotly 대시보드로 결과 시각화하기
3. 세그먼트별 심층 분석하기
4. 비즈니스 의사결정 권장사항 도출하기

---

## 라이브러리 및 데이터 준비

In [ ]:
# 필요한 라이브러리 임포트
import numpy as np
import pandas as pd
from scipy import stats
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

print("라이브러리 임포트 완료")

In [ ]:
# A/B 테스트 데이터 생성 (문제 노트북과 동일)
def generate_ab_test_data(n_users=10000):
    user_ids = [f'user_{str(i).zfill(5)}' for i in range(n_users)]
    groups = np.random.choice(['control', 'treatment'], size=n_users, p=[0.5, 0.5])
    devices = np.random.choice(['mobile', 'desktop'], size=n_users, p=[0.65, 0.35])
    customer_types = np.random.choice(['new', 'returning'], size=n_users, p=[0.4, 0.6])
    payment_methods = np.random.choice(['card', 'easy_pay'], size=n_users, p=[0.45, 0.55])
    start_date = datetime(2025, 1, 1)
    dates = [start_date + timedelta(days=np.random.randint(0, 14)) for _ in range(n_users)]
    
    conversions = []
    order_amounts = []
    page_time_seconds = []
    
    for i in range(n_users):
        if groups[i] == 'control':
            base_rate = 0.12
        else:
            base_rate = 0.145
        
        if devices[i] == 'mobile':
            base_rate *= 0.85
            if groups[i] == 'treatment':
                base_rate *= 1.15
        
        if customer_types[i] == 'returning':
            base_rate *= 1.25
        
        if payment_methods[i] == 'easy_pay':
            base_rate *= 1.1
        
        converted = np.random.random() < base_rate
        conversions.append(1 if converted else 0)
        
        if converted:
            amount = max(10000, np.random.normal(55000, 25000))
            if customer_types[i] == 'returning':
                amount *= 1.15
            order_amounts.append(round(amount, -2))
        else:
            order_amounts.append(0)
        
        if groups[i] == 'control':
            time_spent = max(10, np.random.normal(120, 45))
        else:
            time_spent = max(10, np.random.normal(85, 35))
        page_time_seconds.append(round(time_spent))
    
    df = pd.DataFrame({
        'user_id': user_ids,
        'date': dates,
        'group': groups,
        'device': devices,
        'customer_type': customer_types,
        'payment_method': payment_methods,
        'converted': conversions,
        'order_amount': order_amounts,
        'page_time_seconds': page_time_seconds
    })
    
    return df

df = generate_ab_test_data(n_users=10000)
print(f"데이터 준비 완료: {len(df):,}행")

---

## 🎯 실습 퀴즈 정답

---

### Q1. 표본 크기 계산 ⭐

**문제**: 기존 전환율 8%, 목표 개선율 20% (상대적)일 때, 그룹당 필요한 표본 크기를 계산하세요.

In [ ]:
# Q1 정답

baseline_rate = 0.08
mde = 0.20

# 표본 크기 계산 함수
def calculate_sample_size(baseline_rate, mde, alpha=0.05, power=0.80):
    # 1. 목표 전환율 계산
    target_rate = baseline_rate * (1 + mde)
    
    # 2. 풀링된 표준편차
    pooled_p = (baseline_rate + target_rate) / 2
    pooled_se = np.sqrt(2 * pooled_p * (1 - pooled_p))
    
    # 3. z값 계산
    z_alpha = stats.norm.ppf(1 - alpha/2)
    z_beta = stats.norm.ppf(power)
    
    # 4. 효과 크기
    effect_size = abs(target_rate - baseline_rate)
    
    # 5. 표본 크기 계산
    n = ((z_alpha + z_beta) ** 2 * pooled_se ** 2) / (effect_size ** 2)
    
    return int(np.ceil(n))

# 결과 계산
required_n = calculate_sample_size(baseline_rate, mde)

print(f"기존 전환율: {baseline_rate*100:.1f}%")
print(f"목표 개선율: {mde*100:.0f}% (상대적)")
print(f"목표 전환율: {baseline_rate*(1+mde)*100:.2f}%")
print(f"\n그룹당 필요 표본 크기: {required_n:,}명")
print(f"총 필요 표본 크기: {required_n*2:,}명")

In [ ]:
# 테스트
assert 3000 < required_n < 6000, "표본 크기 범위 확인"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
표본 크기 계산은 통계적 검정력(power)을 확보하기 위해 필요한 최소 샘플 수를 결정합니다.

**핵심 개념**:
- **MDE (Minimum Detectable Effect)**: 감지하려는 최소 효과 크기
- **Power (검정력)**: 실제 효과가 있을 때 이를 탐지할 확률 (보통 80%)
- **Alpha (유의수준)**: 1종 오류 확률 (보통 5%)

**대안 솔루션**:
```python
# statsmodels 라이브러리 사용
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.proportion import proportion_effectsize

effect = proportion_effectsize(0.08, 0.096)
power_analysis = NormalIndPower()
n = power_analysis.solve_power(effect, power=0.8, alpha=0.05, ratio=1)
```

**흔한 실수**:
- MDE를 절대값(0.02)과 상대값(20%)을 혼동
- 단측/양측 검정에 따른 z값 차이 무시

**실무 팁**:
실제로는 예상 이탈률을 고려하여 10-20% 더 많은 표본을 모집합니다.

---

### Q2. SRM 검증 ⭐⭐

**문제**: 다음 그룹 분포가 50:50 할당에서 유의한 편차가 있는지 Chi-square 검정으로 확인하세요.

In [ ]:
# Q2 정답

control_count = 4850
treatment_count = 5150

# 1. 관측값과 기대값 설정
observed = [control_count, treatment_count]
total = control_count + treatment_count
expected = [total / 2, total / 2]  # 50:50 기대

# 2. Chi-square 검정 수행
chi2, p_value = stats.chisquare(observed, expected)

# 3. 결과 출력
print("SRM (Sample Ratio Mismatch) 검증")
print("=" * 50)
print(f"Control: {control_count:,}명 ({control_count/total*100:.1f}%)")
print(f"Treatment: {treatment_count:,}명 ({treatment_count/total*100:.1f}%)")
print(f"\n기대 비율: 50:50")
print(f"실제 비율: {control_count/total*100:.1f}:{treatment_count/total*100:.1f}")
print(f"\nChi-square: {chi2:.4f}")
print(f"p-value: {p_value:.4f}")

# 4. 결론
if p_value > 0.05:
    print("\nSRM 통과: 그룹 할당이 기대대로 수행되었습니다.")
    srm_pass = True
else:
    print("\nSRM 경고: 그룹 할당에 편차가 발생했습니다! 원인 조사 필요.")
    srm_pass = False

In [ ]:
# 테스트
assert chi2 > 0, "Chi-square 값은 양수"
assert 0 <= p_value <= 1, "p-value 범위 확인"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
SRM 검증은 A/B 그룹 할당이 설계대로 되었는지 확인합니다. Chi-square 적합도 검정을 사용합니다.

**핵심 개념**:
- **SRM (Sample Ratio Mismatch)**: 기대 비율과 실제 비율의 불일치
- **Chi-square 적합도 검정**: 관측 빈도와 기대 빈도 비교
- 유의수준 0.05 기준으로 판단

**흔한 실수**:
- SRM 검증 없이 바로 결과 분석 진행
- p-value가 낮아도 무시하고 분석 계속

**실무 팁**:
SRM이 발생하면 실험 결과를 신뢰할 수 없습니다. 원인을 파악하고 데이터를 재수집해야 합니다.

---

### Q3. 전환율 z-검정 ⭐⭐

**문제**: 두 그룹의 전환율 차이가 통계적으로 유의한지 z-검정을 수행하세요.

In [ ]:
# Q3 정답

control_n = 5000
control_conversions = 600
treatment_n = 5000
treatment_conversions = 720

# 1. 전환율 계산
rate_c = control_conversions / control_n
rate_t = treatment_conversions / treatment_n

# 2. 풀링된 비율 계산
pooled_rate = (control_conversions + treatment_conversions) / (control_n + treatment_n)

# 3. 표준오차 계산
pooled_se = np.sqrt(pooled_rate * (1 - pooled_rate) * (1/control_n + 1/treatment_n))

# 4. z-statistic 계산
z_stat = (rate_t - rate_c) / pooled_se

# 5. p-value 계산 (단측: Treatment > Control)
p_value_one_sided = 1 - stats.norm.cdf(z_stat)

# 6. 결과 출력
print("전환율 z-검정 결과")
print("=" * 50)
print(f"Control 전환율: {rate_c*100:.2f}% ({control_conversions}/{control_n})")
print(f"Treatment 전환율: {rate_t*100:.2f}% ({treatment_conversions}/{treatment_n})")
print(f"\n절대 개선: {(rate_t - rate_c)*100:+.2f}%p")
print(f"상대 개선: {(rate_t - rate_c)/rate_c*100:+.1f}%")
print(f"\nz-statistic: {z_stat:.4f}")
print(f"p-value (단측): {p_value_one_sided:.4f}")

# 7. 결론
if p_value_one_sided < 0.05:
    print("\n결론: Treatment가 Control보다 유의하게 높은 전환율을 보입니다!")
else:
    print("\n결론: 통계적으로 유의한 차이가 없습니다.")

In [ ]:
# 테스트
assert abs(rate_c - 0.12) < 0.001, "Control 전환율 확인"
assert abs(rate_t - 0.144) < 0.001, "Treatment 전환율 확인"
assert z_stat > 0, "z-statistic 양수 확인"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
두 비율의 차이를 검정하는 z-test를 사용합니다. 풀링된 비율로 표준오차를 계산합니다.

**핵심 개념**:
- **풀링된 비율**: 두 그룹을 합친 전체 전환율
- **z-statistic**: (차이) / (표준오차)
- **단측 검정**: Treatment > Control 방향만 검정

**대안 솔루션**:
```python
# statsmodels 사용
from statsmodels.stats.proportion import proportions_ztest
count = [720, 600]
nobs = [5000, 5000]
z_stat, p_value = proportions_ztest(count, nobs, alternative='larger')
```

**흔한 실수**:
- 단측/양측 검정 혼동
- 표준오차 계산 시 풀링하지 않음

**실무 팁**:
비즈니스에서는 보통 "개선되었는가?"를 확인하므로 단측 검정이 적절합니다.

---

### Q4. 세그먼트별 전환율 계산 ⭐⭐⭐

**문제**: 우리 실습 데이터(`df`)에서 디바이스와 고객 유형별 조합의 전환율을 피벗 테이블로 계산하세요.

In [ ]:
# Q4 정답

# 1. 피벗 테이블 생성
segment_pivot = df.pivot_table(
    values='converted',
    index='device',
    columns='customer_type',
    aggfunc='mean'
) * 100  # 퍼센트로 변환

# 2. 결과 출력
print("디바이스 x 고객유형별 전환율 (%)")
print("=" * 40)
print(segment_pivot.round(2))

# 3. 인사이트 추가
print("\n인사이트:")
print(f"  - 가장 높은 전환율: desktop + returning ({segment_pivot.loc['desktop', 'returning']:.2f}%)")
print(f"  - 가장 낮은 전환율: mobile + new ({segment_pivot.loc['mobile', 'new']:.2f}%)")

In [ ]:
# 테스트
assert segment_pivot.shape == (2, 2), "2x2 피벗 테이블 확인"
assert segment_pivot.max().max() < 100, "전환율 최대값 확인"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
`pd.pivot_table()`로 두 범주형 변수 조합별 평균 전환율을 계산합니다.

**핵심 개념**:
- **pivot_table**: 행/열 인덱스로 그룹화하여 집계
- **aggfunc='mean'**: 평균 계산 (0과 1의 평균 = 비율)

**대안 솔루션**:
```python
# groupby 사용
df.groupby(['device', 'customer_type'])['converted'].mean().unstack() * 100
```

**흔한 실수**:
- 100을 곱하지 않아 소수점으로 표시
- index/columns 순서 혼동

**실무 팁**:
세그먼트별 분석으로 전체 효과가 없어도 특정 세그먼트에서 큰 효과를 발견할 수 있습니다.

---

### Q5. 95% 신뢰구간 계산 ⭐⭐⭐

**문제**: Treatment 그룹 전환율의 95% 신뢰구간을 계산하세요.

In [ ]:
# Q5 정답

n = 5000
conversions = 720

# 1. 전환율 계산
p = conversions / n

# 2. 표준오차 계산
se = np.sqrt(p * (1 - p) / n)

# 3. 95% 신뢰구간 계산 (z = 1.96)
z = 1.96
ci_lower = p - z * se
ci_upper = p + z * se

# 4. 결과 출력
print("95% 신뢰구간 계산")
print("=" * 50)
print(f"표본 크기: {n:,}명")
print(f"전환 수: {conversions:,}건")
print(f"전환율: {p*100:.2f}%")
print(f"\n표준오차: {se:.4f}")
print(f"\n95% 신뢰구간: [{ci_lower*100:.2f}%, {ci_upper*100:.2f}%]")
print(f"\n해석: 전환율이 {ci_lower*100:.2f}%에서 {ci_upper*100:.2f}% 사이에 있을 확률이 95%입니다.")

In [ ]:
# 테스트
assert abs(p - 0.144) < 0.001, "전환율 확인"
assert ci_lower < p < ci_upper, "신뢰구간 범위 확인"
assert ci_upper - ci_lower < 0.03, "신뢰구간 너비 확인"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
비율의 신뢰구간은 정규분포 근사를 사용하여 p +- z * SE로 계산합니다.

**핵심 개념**:
- **신뢰구간**: 모수가 포함될 것으로 기대되는 구간
- **95% 신뢰수준**: z = 1.96
- **표준오차**: sqrt(p*(1-p)/n)

**대안 솔루션**:
```python
# scipy 사용 (Wilson 신뢰구간)
from statsmodels.stats.proportion import proportion_confint
ci_lower, ci_upper = proportion_confint(conversions, n, alpha=0.05, method='wilson')
```

**흔한 실수**:
- z값으로 1.96 대신 2 사용
- 표준편차와 표준오차 혼동

**실무 팁**:
p-value와 함께 신뢰구간을 보고하면 효과 크기의 불확실성을 전달할 수 있습니다.

---

### Q6. 비즈니스 임팩트 계산 ⭐⭐⭐

**문제**: 전환율이 10%에서 12%로 개선되었을 때, 월간 추가 매출을 계산하세요.

In [ ]:
# Q6 정답

monthly_visitors = 50000
avg_order_value = 45000
old_rate = 0.10
new_rate = 0.12

# 1. 기존 전환 및 매출
old_conversions = monthly_visitors * old_rate
old_revenue = old_conversions * avg_order_value

# 2. 신규 전환 및 매출
new_conversions = monthly_visitors * new_rate
new_revenue = new_conversions * avg_order_value

# 3. 증가분 계산
additional_conversions = new_conversions - old_conversions
additional_revenue = new_revenue - old_revenue

# 4. 결과 출력
print("비즈니스 임팩트 분석")
print("=" * 50)
print(f"\n월간 방문자: {monthly_visitors:,}명")
print(f"평균 주문 금액: {avg_order_value:,}원")
print(f"\n기존 전환율: {old_rate*100:.0f}%")
print(f"신규 전환율: {new_rate*100:.0f}%")
print(f"\n기존 월간 전환: {old_conversions:,.0f}건")
print(f"신규 월간 전환: {new_conversions:,.0f}건")
print(f"추가 전환: +{additional_conversions:,.0f}건")
print(f"\n기존 월간 매출: {old_revenue/1e8:.2f}억원")
print(f"신규 월간 매출: {new_revenue/1e8:.2f}억원")
print(f"추가 매출: +{additional_revenue/1e8:.2f}억원")
print(f"\n연간 추가 매출: +{additional_revenue*12/1e8:.2f}억원")

In [ ]:
# 테스트
assert additional_conversions == 1000, "추가 전환 수 확인"
assert additional_revenue == 45000000, "추가 매출 확인"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
전환율 변화를 매출 임팩트로 환산하여 비즈니스 가치를 정량화합니다.

**핵심 개념**:
- **추가 전환 = 방문자 * (신규 전환율 - 기존 전환율)**
- **추가 매출 = 추가 전환 * 평균 주문금액**

**흔한 실수**:
- 상대적 개선율(20%)을 절대적 개선(2%p)으로 혼동
- 연간 임팩트 계산 시 계절성 미고려

**실무 팁**:
비즈니스 임팩트를 금액으로 보고하면 경영진 설득에 효과적입니다.

---

### Q7. Plotly 막대 그래프 ⭐⭐⭐⭐

**문제**: 세 개 버전(A, B, C)의 전환율을 비교하는 그룹 막대 그래프를 Plotly로 그리세요.

In [ ]:
# Q7 정답

versions = ['Version A', 'Version B', 'Version C']
conversion_rates = [8.5, 10.2, 9.8]

# 1. 막대 그래프 생성
fig = go.Figure()

# 2. 색상 설정 (가장 높은 값 강조)
colors = ['#636EFA', '#EF553B', '#00CC96']
best_idx = conversion_rates.index(max(conversion_rates))

fig.add_trace(go.Bar(
    x=versions,
    y=conversion_rates,
    text=[f"{rate:.1f}%" for rate in conversion_rates],
    textposition='outside',
    marker_color=colors,
    hovertemplate='%{x}<br>전환율: %{y:.1f}%<extra></extra>'
))

# 3. 레이아웃 설정
fig.update_layout(
    title=dict(
        text='버전별 전환율 비교 (A/B/C 테스트)',
        font=dict(size=18)
    ),
    xaxis_title='버전',
    yaxis_title='전환율 (%)',
    yaxis=dict(range=[0, max(conversion_rates) * 1.3]),
    height=450,
    showlegend=False
)

# 4. 최고 버전 주석 추가
fig.add_annotation(
    x=versions[best_idx],
    y=conversion_rates[best_idx] + 0.8,
    text="Best!",
    showarrow=True,
    arrowhead=2,
    font=dict(color='green', size=14)
)

fig.show()

### 풀이 설명

**접근 방법**:
`go.Bar`로 막대 그래프를 생성하고 텍스트, 색상, 주석을 추가합니다.

**핵심 개념**:
- **text**: 막대 위에 값 표시
- **textposition='outside'**: 막대 바깥에 텍스트
- **add_annotation**: 특정 위치에 주석 추가

**대안 솔루션**:
```python
# Plotly Express 사용
fig = px.bar(x=versions, y=conversion_rates, text=conversion_rates,
             title='버전별 전환율 비교')
fig.update_traces(textposition='outside', texttemplate='%{text:.1f}%')
```

**실무 팁**:
경영진 보고 시 가장 좋은 결과를 시각적으로 강조하면 효과적입니다.

---

### Q8. 일별 트렌드 시각화 ⭐⭐⭐⭐

**문제**: 우리 데이터(`df`)에서 그룹별 일별 평균 객단가 트렌드를 선 그래프로 시각화하세요.

In [ ]:
# Q8 정답

# 1. 전환된 사용자만 필터링
df_converted = df[df['converted'] == 1].copy()

# 2. 일별 그룹별 평균 객단가 계산
daily_aov = df_converted.groupby(['date', 'group'])['order_amount'].mean().reset_index()
daily_aov.columns = ['날짜', '그룹', '평균객단가']

# 3. 선 그래프 생성
fig = px.line(
    daily_aov,
    x='날짜',
    y='평균객단가',
    color='그룹',
    title='일별 평균 객단가 트렌드 (전환 고객 기준)',
    labels={'평균객단가': '평균 객단가 (원)'},
    color_discrete_map={'control': '#636EFA', 'treatment': '#EF553B'},
    markers=True
)

# 4. 레이아웃 설정
fig.update_layout(
    height=450,
    yaxis_tickformat=',',
    hovermode='x unified'
)

# 5. 마커 크기 조정
fig.update_traces(marker=dict(size=8))

fig.show()

# 6. 요약 통계
print("\n그룹별 평균 객단가:")
print(df_converted.groupby('group')['order_amount'].mean().apply(lambda x: f"{x:,.0f}원"))

### 풀이 설명

**접근 방법**:
전환된 사용자만 필터링하고, 일별 그룹별로 집계한 후 px.line으로 시각화합니다.

**핵심 개념**:
- 객단가는 전환된 사용자만 대상
- groupby로 일별+그룹별 집계
- markers=True로 데이터 포인트 표시

**흔한 실수**:
- 전환되지 않은 사용자 포함 (order_amount=0)
- 날짜 정렬 누락

**실무 팁**:
트렌드 시각화로 실험 기간 중 이상 패턴(명절 효과 등)을 발견할 수 있습니다.

---

### Q9. 종합 분석 함수 ⭐⭐⭐⭐⭐

**문제**: A/B 테스트 데이터를 받아 전환율 검정, 개선율, 통계적 유의성을 한 번에 분석하는 함수를 작성하세요.

In [ ]:
# Q9 정답

def analyze_ab_test(control_conv, control_n, treatment_conv, treatment_n, alpha=0.05):
    """
    A/B 테스트 종합 분석
    
    Parameters:
    - control_conv: Control 그룹 전환 수
    - control_n: Control 그룹 총 사용자 수
    - treatment_conv: Treatment 그룹 전환 수
    - treatment_n: Treatment 그룹 총 사용자 수
    - alpha: 유의수준 (기본 0.05)
    
    Returns:
    - dict: 분석 결과
    """
    # 1. 전환율 계산
    rate_c = control_conv / control_n
    rate_t = treatment_conv / treatment_n
    
    # 2. 개선율 계산
    absolute_lift = rate_t - rate_c
    relative_lift = (rate_t - rate_c) / rate_c * 100
    
    # 3. 풀링된 비율 및 표준오차
    pooled_rate = (control_conv + treatment_conv) / (control_n + treatment_n)
    pooled_se = np.sqrt(pooled_rate * (1 - pooled_rate) * (1/control_n + 1/treatment_n))
    
    # 4. z-검정
    z_stat = absolute_lift / pooled_se
    p_value_one = 1 - stats.norm.cdf(z_stat)  # 단측
    p_value_two = 2 * (1 - stats.norm.cdf(abs(z_stat)))  # 양측
    
    # 5. 신뢰구간
    se_diff = np.sqrt(rate_c*(1-rate_c)/control_n + rate_t*(1-rate_t)/treatment_n)
    z_crit = stats.norm.ppf(1 - alpha/2)
    ci_lower = absolute_lift - z_crit * se_diff
    ci_upper = absolute_lift + z_crit * se_diff
    
    # 6. 유의성 판정
    significant = p_value_one < alpha
    
    return {
        'control_rate': rate_c,
        'treatment_rate': rate_t,
        'absolute_lift': absolute_lift,
        'relative_lift_pct': relative_lift,
        'z_statistic': z_stat,
        'p_value_one_sided': p_value_one,
        'p_value_two_sided': p_value_two,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'significant': significant
    }

# 테스트 실행
result = analyze_ab_test(600, 5000, 720, 5000)

print("A/B 테스트 분석 결과")
print("=" * 50)
print(f"Control 전환율: {result['control_rate']*100:.2f}%")
print(f"Treatment 전환율: {result['treatment_rate']*100:.2f}%")
print(f"\n절대 개선: {result['absolute_lift']*100:+.2f}%p")
print(f"상대 개선: {result['relative_lift_pct']:+.1f}%")
print(f"\nz-statistic: {result['z_statistic']:.4f}")
print(f"p-value (단측): {result['p_value_one_sided']:.4f}")
print(f"p-value (양측): {result['p_value_two_sided']:.4f}")
print(f"\n95% 신뢰구간: [{result['ci_lower']*100:.2f}%, {result['ci_upper']*100:.2f}%]")
print(f"\n통계적 유의성: {'유의함' if result['significant'] else '유의하지 않음'}")

In [ ]:
# 테스트
assert abs(result['control_rate'] - 0.12) < 0.001
assert abs(result['treatment_rate'] - 0.144) < 0.001
assert result['significant'] == True
assert 'z_statistic' in result
assert 'ci_lower' in result
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
A/B 테스트 분석에 필요한 모든 통계량을 한 함수에서 계산합니다.

**핵심 개념**:
- 전환율, 개선율, z-검정, 신뢰구간, 유의성 판정을 모두 포함
- 단측/양측 p-value 모두 제공
- 딕셔너리로 결과 반환하여 활용성 높임

**흔한 실수**:
- 신뢰구간 계산 시 풀링된 SE 대신 개별 SE 사용
- 결과 키 이름 불일치

**실무 팁**:
이런 함수를 팀 공용 라이브러리로 만들어두면 분석 일관성을 유지할 수 있습니다.

---

### Q10. 전체 파이프라인 프로젝트 ⭐⭐⭐⭐⭐

**문제**: 이메일 마케팅 제목 A/B 테스트 전체 파이프라인을 구현하세요.

In [ ]:
# Q10 정답

print("="*70)
print("이메일 마케팅 제목 A/B 테스트 분석 보고서")
print("="*70)

# ===========================================
# Step 1: 가상 데이터 생성
# ===========================================
np.random.seed(42)

# 각 그룹 3000명, Control 15%, Treatment 18% 오픈율
control_opens = np.random.binomial(1, 0.15, 3000)
treatment_opens = np.random.binomial(1, 0.18, 3000)

# DataFrame 생성
email_df = pd.DataFrame({
    'user_id': range(6000),
    'group': ['control'] * 3000 + ['treatment'] * 3000,
    'opened': list(control_opens) + list(treatment_opens)
})

print("\n1. 실험 설계")
print("-"*50)
print("  실험: 이메일 제목에 이모지 추가 효과 검증")
print("  Control: 기존 제목 (텍스트만)")
print("  Treatment: 이모지 포함 제목")
print(f"  표본 크기: 각 그룹 {len(control_opens):,}명")

In [ ]:
# ===========================================
# Step 2: 기초 통계량 계산
# ===========================================
print("\n2. 기초 통계량")
print("-"*50)

# 그룹별 통계
control_data = email_df[email_df['group'] == 'control']
treatment_data = email_df[email_df['group'] == 'treatment']

control_n = len(control_data)
control_opens_count = control_data['opened'].sum()
control_rate = control_opens_count / control_n

treatment_n = len(treatment_data)
treatment_opens_count = treatment_data['opened'].sum()
treatment_rate = treatment_opens_count / treatment_n

print(f"  Control:   {control_opens_count:,}명 오픈 / {control_n:,}명 = {control_rate*100:.2f}%")
print(f"  Treatment: {treatment_opens_count:,}명 오픈 / {treatment_n:,}명 = {treatment_rate*100:.2f}%")

In [ ]:
# ===========================================
# Step 3: 통계적 검정 수행
# ===========================================
print("\n3. 통계적 검정")
print("-"*50)

# 분석 함수 사용
email_result = analyze_ab_test(
    control_opens_count, control_n,
    treatment_opens_count, treatment_n
)

print(f"  절대 개선: {email_result['absolute_lift']*100:+.2f}%p")
print(f"  상대 개선: {email_result['relative_lift_pct']:+.1f}%")
print(f"  z-statistic: {email_result['z_statistic']:.4f}")
print(f"  p-value (단측): {email_result['p_value_one_sided']:.4f}")
print(f"  95% CI: [{email_result['ci_lower']*100:.2f}%, {email_result['ci_upper']*100:.2f}%]")
print(f"\n  통계적 유의성: {'유의함 (p < 0.05)' if email_result['significant'] else '유의하지 않음'}")

In [ ]:
# ===========================================
# Step 4: Plotly로 결과 시각화
# ===========================================
print("\n4. 결과 시각화")
print("-"*50)

# 서브플롯 생성
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('오픈율 비교', '95% 신뢰구간'),
    column_widths=[0.5, 0.5]
)

# 1) 오픈율 비교 막대 그래프
fig.add_trace(
    go.Bar(
        x=['Control<br>(기존 제목)', 'Treatment<br>(이모지 포함)'],
        y=[control_rate*100, treatment_rate*100],
        text=[f"{control_rate*100:.2f}%", f"{treatment_rate*100:.2f}%"],
        textposition='outside',
        marker_color=['#636EFA', '#EF553B']
    ),
    row=1, col=1
)

# 2) 신뢰구간 시각화
fig.add_trace(
    go.Scatter(
        x=[email_result['absolute_lift']*100],
        y=['개선 효과'],
        mode='markers',
        marker=dict(size=15, color='#00CC96'),
        error_x=dict(
            type='data',
            symmetric=False,
            array=[(email_result['ci_upper'] - email_result['absolute_lift'])*100],
            arrayminus=[(email_result['absolute_lift'] - email_result['ci_lower'])*100]
        ),
        name='개선 효과'
    ),
    row=1, col=2
)

# 0 기준선 추가
fig.add_vline(x=0, line_dash="dash", line_color="gray", row=1, col=2)

fig.update_layout(
    title='이메일 A/B 테스트 결과',
    height=400,
    showlegend=False
)

fig.update_xaxes(title_text='오픈율 (%)', row=1, col=1)
fig.update_xaxes(title_text='개선율 (%p)', row=1, col=2)

fig.show()

In [ ]:
# ===========================================
# Step 5: 비즈니스 권장사항 출력
# ===========================================
print("\n5. 비즈니스 권장사항")
print("-"*50)

# 임팩트 계산 (가정: 월 10만 이메일, 오픈당 200원 가치)
monthly_emails = 100000
value_per_open = 200  # 원

current_opens = monthly_emails * control_rate
expected_opens = monthly_emails * treatment_rate
additional_opens = expected_opens - current_opens
additional_value = additional_opens * value_per_open

print(f"\n  월간 발송량: {monthly_emails:,}건")
print(f"  현재 오픈: {current_opens:,.0f}건")
print(f"  예상 오픈: {expected_opens:,.0f}건 (+{additional_opens:,.0f}건)")
print(f"  예상 월간 추가 가치: {additional_value:,.0f}원")

print("\n" + "="*70)
print("결론 및 권장사항")
print("="*70)

if email_result['significant']:
    print("\n  [권장] 이모지 포함 제목을 전체 캠페인에 적용하세요.")
    print(f"  - 오픈율 {email_result['relative_lift_pct']:.1f}% 개선 예상")
    print(f"  - 월간 {additional_opens:,.0f}건 추가 오픈 예상")
    print("  - 2주 후 효과 재검증 권장")
else:
    print("\n  [보류] 현재 결과로는 변경 권장하지 않습니다.")
    print("  - 추가 테스트 또는 다른 변형 시도 필요")

print("\n" + "="*70)

### 풀이 설명

**접근 방법**:
A/B 테스트 전체 파이프라인을 5단계로 구현합니다.

**핵심 개념**:
1. **데이터 생성**: np.random.binomial으로 이항 분포 시뮬레이션
2. **기초 통계**: 그룹별 전환율 계산
3. **통계 검정**: z-test, 신뢰구간
4. **시각화**: Plotly로 결과 대시보드
5. **권장사항**: 비즈니스 임팩트 기반 의사결정

**흔한 실수**:
- 각 단계 결과를 다음 단계에서 활용하지 않음
- 비즈니스 임팩트 없이 통계 결과만 보고

**실무 팁**:
실제 프로젝트에서는 이 파이프라인을 클래스나 모듈로 만들어 재사용합니다.

---

## 학습 정리

### Part 1: 프로젝트 기획 핵심 요약

| 단계 | 핵심 활동 | 주요 산출물 |
|------|----------|------------|
| 가설 설정 | H0/H1 정의, KPI 선정 | 가설 문서 |
| 실험 설계 | 표본 크기 계산, 기간 결정 | 실험 설계서 |
| 데이터 수집 | 그룹 할당, 지표 트래킹 | 원시 데이터 |
| SRM 검증 | 그룹 비율 검증 | Chi-square 결과 |
| 기초 통계 | 그룹별 요약 통계 | 기술 통계표 |

### Part 2: 분석 및 의사결정 핵심 요약

| 분석 유형 | 검정 방법 | Python 함수 | 활용 상황 |
|----------|----------|-------------|----------|
| 전환율 비교 | z-test (비율) | 직접 구현 | 클릭, 구매 전환 |
| 평균 비교 | t-test | stats.ttest_ind() | 객단가, 체류시간 |
| 범주형 비교 | Chi-square | stats.chisquare() | SRM, 세그먼트 분포 |
| 세그먼트 분석 | 그룹별 검정 | 루프 적용 | 디바이스, 고객 유형 |

### 실무 팁

1. **SRM 검증 필수**: 실험 분석 전 그룹 할당 검증으로 데이터 무결성 확인
2. **표본 크기 사전 계산**: Power Analysis로 실험 기간과 신뢰도 사전 확보
3. **세그먼트별 분석**: 전체 효과가 작아도 특정 세그먼트에서 큰 효과 가능
4. **신뢰구간 보고**: p-value만 아니라 효과 크기의 불확실성도 함께 보고
5. **비즈니스 임팩트 환산**: 통계적 유의성 + 매출 영향으로 의사결정 지원
6. **후속 실험 제안**: 한 번의 실험으로 끝이 아닌 지속적 개선 사이클 구축